# Cluster POROUT

In [120]:
import pandas as pd
import numpy as np
import gc
from datetime import datetime
from itertools import compress
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from joblib import load,dump
import re
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler

In [121]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [10]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)

## CARGA DE DATOS

In [5]:
tablaFinal=pd.read_csv('tabla1_TotalV4.csv')

C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (1,32,35,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Selección variables

In [6]:
varComp=tablaFinal.columns.values

In [7]:
#varNumericas=[]
#varCategoricas=[]
#varEliminar=[]

In [8]:
len(tablaFinal.columns.values)

118

In [8]:
# Manejo de nullos
varEliminar=['CO_ID', 'IDENTIFICACION', 'tipo_id', 'nom_paq_x', 'nom_paq_y']
varCatListas=['tipo_ACT','t_viajero','STD_cat2_PROMEDIO_SALDO','STD_cat2_PROM_SEGUNDOS','STD_cat2_PROM_DATOS','slope_cat2_PROMEDIO_SALDO',
              'slope_cat2_PROM_SEGUNDOS','slope_cat2_PROM_DATOS','res_Region','res_Mun','res_Depto']
varCatTrans=['razon_VOZ Y DATOS','razon_SERVICIO','razon_PQR','razon_PAGO','razon_INFORMACION',
             'razon_FACTURACION','razon_EQUIPO','razon_COMERCIAL','razon_COBRO','razon_CANCELACION',
             'POROUT','paq_youtube','paq_wtf','paq_ws','paq_waze','paq_twitter','paq_todoIncluido','paq_suscripcion',
             'paq_speedy','paq_snapchat','paq_sms','paq_sinpaquete','paq_pasatiempo','paq_minutos','paq_migracion',
             'paq_LargaDistancia','paq_Instagram','paq_facebook','paq_empresas','paq_datos','ES_SMARTPHONE']
varNumericas=['TOTAL_VALOR_DE_ADELANTA_SALDO',
              'TOTAL_MB_YOUTUBE','TOTAL_MB_WHATSAPP','TOTAL_MB_TWITTER_Y_FACEBOOK','TOTAL_MB_SNAPCHAT','TOTAL_MB_INSTAGRAM',
              'TOTAL_MB_GOOGLE_MAPS','STD_PROMEDIO_SALDO','STD_PROM_SEGUNDOS','STD_PROM_DATOS','STD_cat_PROMEDIO_SALDO',
              'STD_cat_PROM_SEGUNDOS','STD_cat_PROM_DATOS','slope_PROMEDIO_SALDO','slope_PROM_SEGUNDOS','slope_PROM_DATOS',
              'slope_cat_PROMEDIO_SALDO','slope_cat_PROM_SEGUNDOS','slope_cat_PROM_DATOS','part_TOTAL_MB_YOUTUBE',
              'part_TOTAL_MB_WHATSAPP','part_TOTAL_MB_TWITTER_Y_FACEBOOK','part_TOTAL_MB_SNAPCHAT','part_TOTAL_MB_INSTAGRAM',
              'part_TOTAL_MB_GOOGLE_MAPS','paq_youtube_v','paq_youtube_q','paq_wtf_v','paq_wtf_q','paq_ws_v','paq_ws_q',
              'paq_waze_v','paq_waze_q','paq_twitter_v','paq_twitter_q','paq_todoIncluido_v','paq_todoIncluido_q',
              'paq_suscripcion_v','paq_suscripcion_q','paq_speedy_v','paq_speedy_q','paq_snapchat_v','paq_snapchat_q',
              'paq_sms_v','paq_sms_q','paq_sinpaquete_v','paq_sinpaquete_q','paq_pasatiempo_v','paq_pasatiempo_q','paq_minutos_v',
              'paq_minutos_q','paq_migracion_v','paq_migracion_q','paq_LargaDistancia_v','paq_LargaDistancia_q','paq_Instagram_v',
              'paq_Instagram_q','paq_facebook_v','paq_facebook_q','paq_empresas_v','paq_empresas_q','paq_datos_v','paq_datos_q',
              'CONSUMO_TOTAL_MB','avg_PROMEDIO_SALDO',
              'avg_PROM_SEGUNDOS','avg_PROM_DATOS','ANTIGUEDAD_USUARIO','ACTIGUEDAD_ACT_BSCS',
              'VLR_PAQ_MAS_COMPRADO_2MESES', 'CANT_PAQ_MAS_COMPRADO_2MESES']

In [10]:
len(varEliminar)

5

In [11]:
len(varCatListas)

11

In [12]:
len(varCatTrans)

31

In [13]:
len(varNumericas)

75

In [14]:
len(varEliminar)+len(varCatListas)+len(varCatTrans)+len(varNumericas)

122

In [15]:
diff_list = list((Counter(varComp) - Counter(varNumericas)).elements())

In [16]:
len(diff_list)

47

In [17]:
diff_list = list((Counter(diff_list) - Counter(varCatTrans)).elements())

In [18]:
len(diff_list)

16

In [19]:
diff_list = list((Counter(diff_list) - Counter(varCatListas)).elements())

In [20]:
len(diff_list)

5

In [21]:
diff_list

['CO_ID', 'IDENTIFICACION', 'tipo_id', 'nom_paq_x', 'nom_paq_y']

In [22]:
tablaFinal.drop(columns=varEliminar, inplace=True)

In [23]:
tablaFinal[varCatListas].isnull().sum()

tipo_ACT                           0
t_viajero                          0
STD_cat2_PROMEDIO_SALDO      5977187
STD_cat2_PROM_SEGUNDOS       4457654
STD_cat2_PROM_DATOS          5077056
slope_cat2_PROMEDIO_SALDO    5977187
slope_cat2_PROM_SEGUNDOS     4457654
slope_cat2_PROM_DATOS        5077056
res_Region                   6363300
res_Mun                      6308415
res_Depto                    6315341
dtype: int64

In [24]:
tablaFinal['t_viajero'].value_counts()

sinInfo        5337623
sinCambio      2625091
interMun        188797
interRegion      69768
interDepto       40852
Name: t_viajero, dtype: int64

In [25]:
for col in varCatListas:
    tablaFinal[col]=tablaFinal[col].astype(str)
    tablaFinal[col]=tablaFinal[col].fillna('sinInfo')

In [26]:
tablaFinal[varCatTrans].isnull().sum()

razon_VOZ Y DATOS           0
razon_SERVICIO              0
razon_PQR                   0
razon_PAGO                  0
razon_INFORMACION           0
razon_FACTURACION           0
razon_EQUIPO                0
razon_COMERCIAL             0
razon_COBRO                 0
razon_CANCELACION           0
POROUT                      0
paq_youtube                 0
paq_wtf                     0
paq_ws                      0
paq_waze                    0
paq_twitter                 0
paq_todoIncluido            0
paq_suscripcion             0
paq_speedy                  0
paq_snapchat                0
paq_sms                     0
paq_sinpaquete              0
paq_pasatiempo              0
paq_minutos                 0
paq_migracion               0
paq_LargaDistancia          0
paq_Instagram               0
paq_facebook                0
paq_empresas                0
paq_datos                   0
ES_SMARTPHONE         4981607
dtype: int64

In [27]:
tablaFinal.dtypes

POROUT                                int64
TOTAL_MB_WHATSAPP                   float64
TOTAL_MB_TWITTER_Y_FACEBOOK         float64
TOTAL_MB_SNAPCHAT                   float64
TOTAL_MB_INSTAGRAM                  float64
TOTAL_MB_YOUTUBE                    float64
TOTAL_MB_GOOGLE_MAPS                float64
CONSUMO_TOTAL_MB                    float64
ANTIGUEDAD_USUARIO                  float64
TOTAL_VALOR_DE_ADELANTA_SALDO       float64
ES_SMARTPHONE                       float64
ACTIGUEDAD_ACT_BSCS                   int64
avg_PROM_SEGUNDOS                   float64
STD_PROM_SEGUNDOS                   float64
STD_cat_PROM_SEGUNDOS               float64
STD_cat2_PROM_SEGUNDOS               object
slope_PROM_SEGUNDOS                 float64
slope_cat_PROM_SEGUNDOS             float64
slope_cat2_PROM_SEGUNDOS             object
avg_PROM_DATOS                      float64
STD_PROM_DATOS                      float64
STD_cat_PROM_DATOS                  float64
STD_cat2_PROM_DATOS             

In [29]:
for col in varCatTrans:    
    print(col)
    tablaFinal[col]=tablaFinal[col].fillna('sinInfo')
    tablaFinal[col]=tablaFinal[col].astype(str)

razon_VOZ Y DATOS
razon_SERVICIO
razon_PQR
razon_PAGO
razon_INFORMACION
razon_FACTURACION
razon_EQUIPO
razon_COMERCIAL
razon_COBRO
razon_CANCELACION
POROUT
paq_youtube
paq_wtf
paq_ws
paq_waze
paq_twitter
paq_todoIncluido
paq_suscripcion
paq_speedy
paq_snapchat
paq_sms
paq_sinpaquete
paq_pasatiempo
paq_minutos
paq_migracion
paq_LargaDistancia
paq_Instagram
paq_facebook
paq_empresas
paq_datos
ES_SMARTPHONE


In [34]:
dump(tablaFinal,'tablaFormats.pkl')

['tablaFormats.pkl']

In [4]:
tablaFinal=load('tablaFormats.pkl')

In [7]:
tablaFinal.head()

,POROUT,TOTAL_MB_WHATSAPP,TOTAL_MB_TWITTER_Y_FACEBOOK,TOTAL_MB_SNAPCHAT,TOTAL_MB_INSTAGRAM,TOTAL_MB_YOUTUBE,TOTAL_MB_GOOGLE_MAPS,CONSUMO_TOTAL_MB,ANTIGUEDAD_USUARIO,TOTAL_VALOR_DE_ADELANTA_SALDO,...,razon_COMERCIAL,razon_EQUIPO,razon_FACTURACION,razon_INFORMACION,razon_PAGO,razon_PQR,razon_SERVICIO,razon_VOZ Y DATOS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES
0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0.0
1,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,NaN,NaN,...,0,0,0,0,0,0,1,0,0.0,0.0
3,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,...,0,0,0,1,0,0,0,0,0.0,0.0
4,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0.0


In [11]:
tablaFinal[varNumericas].isnull().sum()

TOTAL_VALOR_DE_ADELANTA_SALDO       5688616
TOTAL_MB_YOUTUBE                          0
TOTAL_MB_WHATSAPP                         0
TOTAL_MB_TWITTER_Y_FACEBOOK               0
TOTAL_MB_SNAPCHAT                         0
TOTAL_MB_INSTAGRAM                        0
TOTAL_MB_GOOGLE_MAPS                      0
STD_PROMEDIO_SALDO                  5282562
STD_PROM_SEGUNDOS                   4457634
STD_PROM_DATOS                      5001166
STD_cat_PROMEDIO_SALDO              5977187
STD_cat_PROM_SEGUNDOS               4457654
STD_cat_PROM_DATOS                  5077056
slope_PROMEDIO_SALDO                5282562
slope_PROM_SEGUNDOS                 4457634
slope_PROM_DATOS                    5001166
slope_cat_PROMEDIO_SALDO            5977187
slope_cat_PROM_SEGUNDOS             4457654
slope_cat_PROM_DATOS                5077056
part_TOTAL_MB_YOUTUBE                     0
part_TOTAL_MB_WHATSAPP                    0
part_TOTAL_MB_TWITTER_Y_FACEBOOK          0
part_TOTAL_MB_SNAPCHAT          

In [12]:
tablaFinal[varNumericas].dtypes

TOTAL_VALOR_DE_ADELANTA_SALDO       float64
TOTAL_MB_YOUTUBE                    float64
TOTAL_MB_WHATSAPP                   float64
TOTAL_MB_TWITTER_Y_FACEBOOK         float64
TOTAL_MB_SNAPCHAT                   float64
TOTAL_MB_INSTAGRAM                  float64
TOTAL_MB_GOOGLE_MAPS                float64
STD_PROMEDIO_SALDO                  float64
STD_PROM_SEGUNDOS                   float64
STD_PROM_DATOS                      float64
STD_cat_PROMEDIO_SALDO              float64
STD_cat_PROM_SEGUNDOS               float64
STD_cat_PROM_DATOS                  float64
slope_PROMEDIO_SALDO                float64
slope_PROM_SEGUNDOS                 float64
slope_PROM_DATOS                    float64
slope_cat_PROMEDIO_SALDO            float64
slope_cat_PROM_SEGUNDOS             float64
slope_cat_PROM_DATOS                float64
part_TOTAL_MB_YOUTUBE               float64
part_TOTAL_MB_WHATSAPP              float64
part_TOTAL_MB_TWITTER_Y_FACEBOOK    float64
part_TOTAL_MB_SNAPCHAT          

In [22]:
varFillCero=['avg_PROMEDIO_SALDO','avg_PROM_SEGUNDOS','avg_PROM_DATOS','TOTAL_VALOR_DE_ADELANTA_SALDO']
varFillMedian=['ANTIGUEDAD_USUARIO','STD_PROMEDIO_SALDO','STD_PROM_SEGUNDOS','STD_PROM_DATOS',
              'STD_cat_PROMEDIO_SALDO','STD_cat_PROM_SEGUNDOS','STD_cat_PROM_DATOS','slope_PROMEDIO_SALDO',
              'slope_PROM_SEGUNDOS','slope_PROM_DATOS','slope_cat_PROMEDIO_SALDO','slope_cat_PROM_SEGUNDOS',
              'slope_cat_PROM_DATOS']

In [21]:
for col in varFillCero:    
    print(col)
    tablaFinal[col]=tablaFinal[col].fillna(0)

avg_PROMEDIO_SALDO
avg_PROM_SEGUNDOS
avg_PROM_DATOS
TOTAL_VALOR_DE_ADELANTA_SALDO


In [24]:
for col in varFillMedian:    
    print(col)
    auxMedian=tablaFinal[col].median()
    print(auxMedian)
    tablaFinal[col]=tablaFinal[col].fillna(auxMedian)

ANTIGUEDAD_USUARIO
435.0
STD_PROMEDIO_SALDO
74.73275552498922
STD_PROM_SEGUNDOS
77.30722886563017
STD_PROM_DATOS
857414.5455734777
STD_cat_PROMEDIO_SALDO
0.8199270583239278
STD_cat_PROM_SEGUNDOS
0.3629747756156358
STD_cat_PROM_DATOS
0.8468910606287521
slope_PROMEDIO_SALDO
0.0
slope_PROM_SEGUNDOS
-1.2499999999999971
slope_PROM_DATOS
0.0
slope_cat_PROMEDIO_SALDO
-1.6081192895528891e-16
slope_cat_PROM_SEGUNDOS
-0.0072424963710119
slope_cat_PROM_DATOS
0.0001955820843275


###  Ordenar columnas

In [26]:
tablaFinal=tablaFinal[varCatListas + varCatTrans + varNumericas]

In [54]:
varCategoricas=varCatListas + varCatTrans

In [27]:
tablaFinal.dtypes

tipo_ACT                             object
t_viajero                            object
STD_cat2_PROMEDIO_SALDO              object
STD_cat2_PROM_SEGUNDOS               object
STD_cat2_PROM_DATOS                  object
slope_cat2_PROMEDIO_SALDO            object
slope_cat2_PROM_SEGUNDOS             object
slope_cat2_PROM_DATOS                object
res_Region                           object
res_Mun                              object
res_Depto                            object
razon_VOZ Y DATOS                    object
razon_SERVICIO                       object
razon_PQR                            object
razon_PAGO                           object
razon_INFORMACION                    object
razon_FACTURACION                    object
razon_EQUIPO                         object
razon_COMERCIAL                      object
razon_COBRO                          object
razon_CANCELACION                    object
POROUT                               object
paq_youtube                     

## Cluster

In [32]:
cat_idx=np.arange(0,len(varCatListas)+len(varCatTrans),1)

In [41]:
cat_idx=list(cat_idx)

In [36]:
kproto = KPrototypes(n_clusters=5, verbose=2, max_iter=20)

### Ejercicio1
* Sample 1000 casos para validar resultados

In [37]:
tablaFinalSample=tablaFinal.sample(1000)

In [42]:
#kproto.fit(df_array, categorical=cat_idx)
clusters=kproto.fit(tablaFinalSample, categorical=cat_idx)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 37, ncost: 2.0404483699291156e+16
Run: 1, iteration: 2/20, moves: 12, ncost: 1.9983579641154716e+16
Run: 1, iteration: 3/20, moves: 3, ncost: 1.974201531771256e+16
Run: 1, iteration: 4/20, moves: 4, ncost: 1.9150823753042228e+16
Run: 1, iteration: 5/20, moves: 12, ncost: 1.725007798946048e+16
Run: 1, iteration: 6/20, moves: 8, ncost: 1.6918973320048154e+16
Run: 1, iteration: 7/20, moves: 1, ncost: 1.6918010007028246e+16
Run: 1, iteration: 8/20, moves: 0, ncost: 1.6918010007028246e+16
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/20, moves: 114, ncost: 2.0597969500332236e+16
Run: 2, iteration: 2/20, moves: 40, ncost: 1.9016283200213376e+16
Run: 2, iteration: 3/20, moves: 22, ncost: 1.8073579067466204e+16
Run: 2, iteration: 4/20, moves: 19, ncost: 1.7341808124286478e+

Starting iterations...
Run: 9, iteration: 1/20, moves: 105, ncost: 2.1943410394024532e+16
Run: 9, iteration: 2/20, moves: 39, ncost: 1.9569455200506012e+16
Run: 9, iteration: 3/20, moves: 7, ncost: 1.9470834520020476e+16
Run: 9, iteration: 4/20, moves: 2, ncost: 1.9462813996815204e+16
Run: 9, iteration: 5/20, moves: 2, ncost: 1.9452583575332264e+16
Run: 9, iteration: 6/20, moves: 1, ncost: 1.9449645656546096e+16
Run: 9, iteration: 7/20, moves: 0, ncost: 1.9449645656546096e+16
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 10, iteration: 1/20, moves: 139, ncost: 2.4140566626764932e+16
Run: 10, iteration: 2/20, moves: 63, ncost: 2.063104019121603e+16
Run: 10, iteration: 

In [43]:
print(kproto.cluster_centroids_)

[['62.5' '1205.25' '3433.25' '12715.5' '55.5' '3.75' '5.625'
  '241.89912790403145' '84.94873025316927' '5641438.021924371'
  '0.898893165896152' '0.27871033170323956' '0.40709309927120374'
  '-194.68790325000023' '-16.62513657499997' '1947661.2334794442'
  '-0.30261169829814183' '-0.02035894409725797' '0.0328973312671618'
  '0.04286697188038474' '0.13443415557979707' '0.45926198829052567'
  '0.0025976678888821507' '0.00011707287744678468'
  '0.0003695953958023296' '0.0' '0.0' '-250.0' '0.125' '-500.0' '0.25'
  '0.0' '0.0' '-500.0' '0.25' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0'
  '0.0' '0.0' '0.0' '-625.0' '0.375' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0'
  '0.0' '0.0' '0.0' '0.0' '-500.0' '0.25' '0.0' '0.0' '-625.0' '0.375'
  '20088.0' '269.97151875' '555.2488168979157' '63942925.31529833'
  '415.0' '664.0' '-625.0' '0.375' 'seMantiene' 'sinCambio' 'nan'
  'moderate' 'united' 'nan' 'noTrend' 'noTrend' 'nan' 'nan' 'nan' '0' '0'
  '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' 

In [45]:
clusters

KPrototypes(gamma=563253.502255537, max_iter=20, n_clusters=5, verbose=2)

In [46]:
"""cluster_dict=[]
for c in clusters:
    cluster_dict.append(c)"""

'cluster_dict=[]\nfor c in clusters:\n    cluster_dict.append(c)'

In [47]:
#df['Cluster Labels'] = kprototype.labels_
tablaFinalSample['cluster']=clusters.labels_

In [48]:
tablaFinalSample.sample(20)

,tipo_ACT,t_viajero,STD_cat2_PROMEDIO_SALDO,STD_cat2_PROM_SEGUNDOS,STD_cat2_PROM_DATOS,slope_cat2_PROMEDIO_SALDO,slope_cat2_PROM_SEGUNDOS,slope_cat2_PROM_DATOS,res_Region,res_Mun,res_Depto,razon_VOZ Y DATOS,razon_SERVICIO,razon_PQR,razon_PAGO,razon_INFORMACION,razon_FACTURACION,razon_EQUIPO,razon_COMERCIAL,razon_COBRO,razon_CANCELACION,POROUT,paq_youtube,paq_wtf,paq_ws,paq_waze,paq_twitter,paq_todoIncluido,paq_suscripcion,paq_speedy,paq_snapchat,paq_sms,paq_sinpaquete,paq_pasatiempo,paq_minutos,paq_migracion,paq_LargaDistancia,paq_Instagram,paq_facebook,paq_empresas,paq_datos,ES_SMARTPHONE,TOTAL_VALOR_DE_ADELANTA_SALDO,TOTAL_MB_YOUTUBE,TOTAL_MB_WHATSAPP,TOTAL_MB_TWITTER_Y_FACEBOOK,TOTAL_MB_SNAPCHAT,TOTAL_MB_INSTAGRAM,TOTAL_MB_GOOGLE_MAPS,STD_PROMEDIO_SALDO,STD_PROM_SEGUNDOS,STD_PROM_DATOS,STD_cat_PROMEDIO_SALDO,STD_cat_PROM_SEGUNDOS,STD_cat_PROM_DATOS,slope_PROMEDIO_SALDO,slope_PROM_SEGUNDOS,slope_PROM_DATOS,slope_cat_PROMEDIO_SALDO,slope_cat_PROM_SEGUNDOS,slope_cat_PROM_DATOS,part_TOTAL_MB_YOUTUBE,part_TOTAL_MB_WHATSAPP,part_TOTAL_MB_TWITTER_Y_FACEBOOK,part_TOTAL_MB_SNAPCHAT,part_TOTAL_MB_INSTAGRAM,part_TOTAL_MB_GOOGLE_MAPS,paq_youtube_v,paq_youtube_q,paq_wtf_v,paq_wtf_q,paq_ws_v,paq_ws_q,paq_waze_v,paq_waze_q,paq_twitter_v,paq_twitter_q,paq_todoIncluido_v,paq_todoIncluido_q,paq_suscripcion_v,paq_suscripcion_q,paq_speedy_v,paq_speedy_q,paq_snapchat_v,paq_snapchat_q,paq_sms_v,paq_sms_q,paq_sinpaquete_v,paq_sinpaquete_q,paq_pasatiempo_v,paq_pasatiempo_q,paq_minutos_v,paq_minutos_q,paq_migracion_v,paq_migracion_q,paq_LargaDistancia_v,paq_LargaDistancia_q,paq_Instagram_v,paq_Instagram_q,paq_facebook_v,paq_facebook_q,paq_empresas_v,paq_empresas_q,paq_datos_v,paq_datos_q,CONSUMO_TOTAL_MB,avg_PROMEDIO_SALDO,avg_PROM_SEGUNDOS,avg_PROM_DATOS,ANTIGUEDAD_USUARIO,ACTIGUEDAD_ACT_BSCS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES,cluster
3601769,seMantiene,interRegion,dispersed,dispersed,moderate,increase,noTrend,noTrend,SurOccidente,Soacha,Caqueta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,952.176,175.883,38040.539,1.414,0.529,0.934,1346.581,-24.131,2172.078,2.000,-0.073,0.053,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9.000,673.290,332.374,40746.955,136.000,295,0.000,0.000,4
5842160,seMantiene,sinInfo,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,74.733,77.307,857414.546,0.820,0.363,0.847,0.000,-1.250,0.000,-0.000,-0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1.000,133.333,0.000,0.000,1.000,137,0.000,0.000,4
2288702,seMantiene,sinCambio,united,moderate,united,noTrend,noTrend,noTrend,Oriente,Aguada,Santander,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.000,103.000,993.000,7745.000,1.000,2501.000,29.000,0.000,93.728,2468644.048,0.000,0.454,0.273,-0.000,-18.815,778575.972,-0.000,-0.091,0.086,0.006,0.054,0.418,0.000,0.135,0.002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,18512.000,1.000,206.674,9042682.969,547.000,580,0.000,0.000,2
3784936,seMantiene,sinCambio,nan,moderate,nan,nan,decrease,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,sinInfo,0.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,74.733,43.970,857414.546,0.820,0.363,0.847,0.000,-31.847,0.000,-0.000,-0.263,0.000,0.000,0.000,0.000,0.000

In [55]:
varCategoricas+['cluster']

['tipo_ACT',
 't_viajero',
 'STD_cat2_PROMEDIO_SALDO',
 'STD_cat2_PROM_SEGUNDOS',
 'STD_cat2_PROM_DATOS',
 'slope_cat2_PROMEDIO_SALDO',
 'slope_cat2_PROM_SEGUNDOS',
 'slope_cat2_PROM_DATOS',
 'res_Region',
 'res_Mun',
 'res_Depto',
 'razon_VOZ Y DATOS',
 'razon_SERVICIO',
 'razon_PQR',
 'razon_PAGO',
 'razon_INFORMACION',
 'razon_FACTURACION',
 'razon_EQUIPO',
 'razon_COMERCIAL',
 'razon_COBRO',
 'razon_CANCELACION',
 'POROUT',
 'paq_youtube',
 'paq_wtf',
 'paq_ws',
 'paq_waze',
 'paq_twitter',
 'paq_todoIncluido',
 'paq_suscripcion',
 'paq_speedy',
 'paq_snapchat',
 'paq_sms',
 'paq_sinpaquete',
 'paq_pasatiempo',
 'paq_minutos',
 'paq_migracion',
 'paq_LargaDistancia',
 'paq_Instagram',
 'paq_facebook',
 'paq_empresas',
 'paq_datos',
 'ES_SMARTPHONE',
 'cluster']

In [56]:
tablaFinalSample[varCategoricas+['cluster']].groupby('cluster').agg(lambda x:x.value_counts().index[0])

,tipo_ACT,t_viajero,STD_cat2_PROMEDIO_SALDO,STD_cat2_PROM_SEGUNDOS,STD_cat2_PROM_DATOS,slope_cat2_PROMEDIO_SALDO,slope_cat2_PROM_SEGUNDOS,slope_cat2_PROM_DATOS,res_Region,res_Mun,res_Depto,razon_VOZ Y DATOS,razon_SERVICIO,razon_PQR,razon_PAGO,razon_INFORMACION,razon_FACTURACION,razon_EQUIPO,razon_COMERCIAL,razon_COBRO,razon_CANCELACION,POROUT,paq_youtube,paq_wtf,paq_ws,paq_waze,paq_twitter,paq_todoIncluido,paq_suscripcion,paq_speedy,paq_snapchat,paq_sms,paq_sinpaquete,paq_pasatiempo,paq_minutos,paq_migracion,paq_LargaDistancia,paq_Instagram,paq_facebook,paq_empresas,paq_datos,ES_SMARTPHONE
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,seMantiene,sinCambio,nan,moderate,united,nan,noTrend,noTrend,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
1,seMantiene,sinCambio,nan,united,united,nan,decrease,decrease,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
2,seMantiene,sinCambio,nan,united,united,nan,noTrend,increase,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
3,seMantiene,sinCambio,dispersed,moderate,moderate,increase,increase,increase,NorOccidente,Manizales,Santander,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
4,seMantiene,sinInfo,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,sinInfo


In [57]:
tablaFinalSample[varNumericas+['cluster']].groupby('cluster').mean()

,TOTAL_VALOR_DE_ADELANTA_SALDO,TOTAL_MB_YOUTUBE,TOTAL_MB_WHATSAPP,TOTAL_MB_TWITTER_Y_FACEBOOK,TOTAL_MB_SNAPCHAT,TOTAL_MB_INSTAGRAM,TOTAL_MB_GOOGLE_MAPS,STD_PROMEDIO_SALDO,STD_PROM_SEGUNDOS,STD_PROM_DATOS,STD_cat_PROMEDIO_SALDO,STD_cat_PROM_SEGUNDOS,STD_cat_PROM_DATOS,slope_PROMEDIO_SALDO,slope_PROM_SEGUNDOS,slope_PROM_DATOS,slope_cat_PROMEDIO_SALDO,slope_cat_PROM_SEGUNDOS,slope_cat_PROM_DATOS,part_TOTAL_MB_YOUTUBE,part_TOTAL_MB_WHATSAPP,part_TOTAL_MB_TWITTER_Y_FACEBOOK,part_TOTAL_MB_SNAPCHAT,part_TOTAL_MB_INSTAGRAM,part_TOTAL_MB_GOOGLE_MAPS,paq_youtube_v,paq_youtube_q,paq_wtf_v,paq_wtf_q,paq_ws_v,paq_ws_q,paq_waze_v,paq_waze_q,paq_twitter_v,paq_twitter_q,paq_todoIncluido_v,paq_todoIncluido_q,paq_suscripcion_v,paq_suscripcion_q,paq_speedy_v,paq_speedy_q,paq_snapchat_v,paq_snapchat_q,paq_sms_v,paq_sms_q,paq_sinpaquete_v,paq_sinpaquete_q,paq_pasatiempo_v,paq_pasatiempo_q,paq_minutos_v,paq_minutos_q,paq_migracion_v,paq_migracion_q,paq_LargaDistancia_v,paq_LargaDistancia_q,paq_Instagram_v,paq_Instagram_q,paq_facebook_v,paq_facebook_q,paq_empresas_v,paq_empresas_q,paq_datos_v,paq_datos_q,CONSUMO_TOTAL_MB,avg_PROMEDIO_SALDO,avg_PROM_SEGUNDOS,avg_PROM_DATOS,ANTIGUEDAD_USUARIO,ACTIGUEDAD_ACT_BSCS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,62.500,1205.250,3433.250,12715.500,55.500,3.750,5.625,241.899,84.949,5641438.022,0.899,0.279,0.407,-194.688,-16.625,1947661.233,-0.303,-0.020,0.033,0.043,0.134,0.459,0.003,0.000,0.000,0.000,0.000,-250.000,0.125,-500.000,0.250,0.000,0.000,-500.000,0.250,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-625.000,0.375,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-500.000,0.250,0.000,0.000,-625.000,0.375,20088.000,269.972,555.249,63942925.315,415.000,664.000,-625.000,0.375
1,25.000,725.400,1218.300,5280.100,63.450,98.100,-0.300,155.415,185.672,12663630.916,0.984,0.341,0.550,38.935,-3.999,-9503391.548,-0.054,-0.013,-0.462,0.048,0.116,0.341,0.002,0.005,0.000,0.000,0.000,-450.000,0.100,-1600.000,0.700,0.000,0.000,-1600.000,0.700,0.000,0.000,0.000,0.000,-292.950,0.200,0.000,0.000,0.000,0.000,-1950.000,0.700,0.000,0.000,-300.000,0.250,-300.000,0.250,0.000,0.000,0.000,0.000,-1600.000,0.700,0.000,0.000,-2392.950,1.000,9189.450,115.034,535.185,27106562.191,854.250,1001.700,-2542.950,1.150
2,81.522,383.880,943.043,3317.000,25.978,71.196,17.587,232.630,119.613,5575955.679,0.775,0.386,0.686,14.668,3.281,1774708.454,-0.028,0.005,0.231,0.050,0.133,0.370,0.003,0.007,0.003,0.000,0.000,-152.174,0.087,-706.522,0.337,0.000,0.000,-684.783,0.326,0.000,0.000,0.000,0.000,-184.902,0.098,0.000,0.000,0.000,0.000,-614.250,0.315,0.000,0.000,-10.870,0.011,-10.870,0.011,0.000,0.000,0.000,0.000,-684.783,0.326,0.000,0.000,-983.696,0.511,6551.620,369.279,347.992,9852260.521,573.815,670.804,-1076.207,0.565
3,0.000,1295.333,637.333,17349.667,65.000,76.000,0.667,130.176,99.718,29335421.381,1.145,0.383,1.000,102.387,35.198,36664067.570,0.389,0.138,1.241,0.044,0.028,0.792,0.004,0.005,0.000,0.000,0.000,-666.667,0.333,-666.667,0.333,0.000,0.000,-666.667,0.333,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-666.667,0.333,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-666.667,0.333,0.000,0.000,-666.667,0.333,22222.667,96.892,185.161,29551498.710,61.333,132.333,-666.667,0.333
4,12.828,19.828,34.737,70.445,0.217,1.198,0.035,253.699,89.711,838323.374,0.808,0.389,0.888,59.694,-3.149,993.037,0.009,-0.016,0.001,0.012,0.026,0.037,0.001,0.001,0.000,0.000,0.000,-18.244,0.009,-176.739,0.095,0.000,0.000,-144.812,0.064,0.000,0.000,0.000,0.000,-34.485,0.026,0.000,0.000,0.000,0.000,-134.645,0.082,0.000,0.000,-85.519,0.070,-85.519,0.070,0.000,0.000,0.000,0.000,-144.812,0.064,0.000,0.000,-239.453,0.154,205.717,521.410,135.819,434541.864,520.940,650.749,-345.773,0.242


In [61]:
tablaFinalSample['cluster'].value_counts()

4    877
2     92
1     20
0      8
3      3
Name: cluster, dtype: int64

In [ ]:
### eliminar columnas
### Selec filas sin servicios

## Ejercicio 2
* Sample con Selección de columnas

#### Seleccion Variables Categoricas

In [65]:
varSelecCat=['t_viajero','STD_cat2_PROMEDIO_SALDO', 'STD_cat2_PROM_SEGUNDOS',
 'STD_cat2_PROM_DATOS', 'slope_cat2_PROMEDIO_SALDO', 'slope_cat2_PROM_SEGUNDOS', 'slope_cat2_PROM_DATOS',
  'razon_COMERCIAL', 'razon_COBRO', 'razon_CANCELACION', 'razon_INFORMACION' ,'POROUT','res_Region']

In [66]:
varSelecNum=['part_TOTAL_MB_YOUTUBE', 'part_TOTAL_MB_WHATSAPP', 'part_TOTAL_MB_TWITTER_Y_FACEBOOK',
 'part_TOTAL_MB_SNAPCHAT', 'part_TOTAL_MB_INSTAGRAM', 'part_TOTAL_MB_GOOGLE_MAPS',
  'CONSUMO_TOTAL_MB', 'avg_PROMEDIO_SALDO', 'avg_PROM_SEGUNDOS', 'avg_PROM_DATOS',
 'paq_youtube_q', 'paq_ws_q', 'paq_waze_q',  'paq_speedy_q', 'paq_snapchat_q', 'paq_minutos_q',
 'paq_migracion_q', 'paq_Instagram_q', 'paq_facebook_q', 'paq_datos_q','ANTIGUEDAD_USUARIO',
 'ACTIGUEDAD_ACT_BSCS', 'VLR_PAQ_MAS_COMPRADO_2MESES', 'CANT_PAQ_MAS_COMPRADO_2MESES'       
            ]

In [67]:
tablaFinalSample=tablaFinal.sample(1000)

In [68]:
tablaFinalSample=tablaFinalSample[varSelecCat+varSelecNum]

In [71]:
cat_idx=np.arange(0,len(varSelecCat),1)
cat_idx=list(cat_idx)

In [72]:
kproto = KPrototypes(n_clusters=5, verbose=2, max_iter=20)
clusters=kproto.fit(tablaFinalSample, categorical=cat_idx)

Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 46, ncost: 7394681085691180.0
Run

Run: 8, iteration: 6/20, moves: 8, ncost: 3303954082173545.0
Run: 8, iteration: 7/20, moves: 5, ncost: 3279529333802016.5
Run: 8, iteration: 8/20, moves: 1, ncost: 3278619391561439.5
Run: 8, iteration: 9/20, moves: 0, ncost: 3278619391561439.5
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 9, iteration: 1/20, moves: 55, ncost: 6303037213146398.0
Run: 9, iteration: 2/20, moves: 37, ncost: 5176042404687388.0
Run: 9, iteration: 3/20, moves: 20, ncost: 4253214356177345.0
Run: 9, iteration: 4/20, moves: 14, ncost: 3671975444166899.5
Run: 9, iteration: 5/20, moves: 21, ncost: 3432011749144621.0
Run: 9, iteration: 6/20, moves: 16, ncost: 3331382994859240.0
Run: 9, iteration: 7/20, moves: 9, ncost: 3304973670345233.0
Run: 9, iteration: 8/20, moves: 5, ncost: 3280173010953011.5
Run: 9, iteration: 9/20, moves: 3, ncost: 3278619391561439.5
Run: 9, iteration: 10/20, moves: 0, ncost: 3278619391561439.5
Init: initializing centroids
Init: initializing clusters
In

In [73]:
print(kproto.cluster_centroids_)

[['0.06751896822166605' '0.12167450064013885' '0.3128588406287975'
  '0.0046633353115736395' '0.014068215918009093' '0.010250978693211352'
  '3553.546296296296' '132.46834096296294' '342.42963279583313'
  '5917013.0805452885' '0.0' '0.4166666666666667' '0.0'
  '0.1111111111111111' '0.0' '0.21296296296296297' '0.21296296296296297'
  '0.0' '0.37962962962962965' '0.5' '650.5462962962963'
  '763.175925925926' '-1250.712962962963' '0.75' 'sinCambio' 'nan'
  'moderate' 'moderate' 'nan' 'noTrend' 'increase' '0' '0' '0' '0' '0'
  'nan']
 ['0.007568552083477313' '0.01670172422864799' '0.023935496798395732'
  '0.0006728711789972052' '0.00052637242579821' '5.893425905376557e-05'
  '93.87620192307692' '353.82129021995166' '116.93611812960738'
  '190041.00952879243' '0.0' '0.027644230769230768' '0.0' '0.015625'
  '0.0' '0.07451923076923077' '0.07451923076923077' '0.0'
  '0.02283653846153846' '0.042067307692307696' '482.47836538461536'
  '553.78125' '-211.40865384615384' '0.13221153846153846' 'sinIn

In [75]:
tablaFinalSample['cluster']=clusters.labels_
tablaFinalSample.sample(5)

,t_viajero,STD_cat2_PROMEDIO_SALDO,STD_cat2_PROM_SEGUNDOS,STD_cat2_PROM_DATOS,slope_cat2_PROMEDIO_SALDO,slope_cat2_PROM_SEGUNDOS,slope_cat2_PROM_DATOS,razon_COMERCIAL,razon_COBRO,razon_CANCELACION,razon_INFORMACION,POROUT,res_Region,part_TOTAL_MB_YOUTUBE,part_TOTAL_MB_WHATSAPP,part_TOTAL_MB_TWITTER_Y_FACEBOOK,part_TOTAL_MB_SNAPCHAT,part_TOTAL_MB_INSTAGRAM,part_TOTAL_MB_GOOGLE_MAPS,CONSUMO_TOTAL_MB,avg_PROMEDIO_SALDO,avg_PROM_SEGUNDOS,avg_PROM_DATOS,paq_youtube_q,paq_ws_q,paq_waze_q,paq_speedy_q,paq_snapchat_q,paq_minutos_q,paq_migracion_q,paq_Instagram_q,paq_facebook_q,paq_datos_q,ANTIGUEDAD_USUARIO,ACTIGUEDAD_ACT_BSCS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES,cluster
208069,sinInfo,nan,nan,nan,nan,nan,nan,0,0,0,0,0,nan,0.000,0.000,0.000,0.000,0.000,0.000,-1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,435.000,2689,0.000,0.000,1
2097327,sinInfo,nan,nan,nan,nan,nan,nan,0,0,0,0,0,nan,0.000,0.000,0.000,0.000,0.000,0.000,-1.000,0.000,520.667,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,435.000,641,0.000,0.000,1
6392851,sinInfo,nan,nan,nan,nan,nan,nan,0,0,0,0,0,nan,0.000,0.000,0.000,0.000,0.000,0.000,-1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,435.000,100,0.000,0.000,1
5395603,sinInfo,nan,nan,dispersed,nan,nan,increase,0,0,0,0,0,nan,0.000,0.000,0.000,0.000,0.000,0.000,-1.000,0.000,0.000,158.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,435.000,162,0.000,0.000,1
920760,sinInfo,nan,moderate,dispersed,nan,decrease,decrease,0,0,0,0,0,nan,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,123.720,141864.279,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1193.000,1260,0.000,0.000,1


In [ ]:
varSelecCat+varSelecNum

In [77]:
tablaFinalSample[varSelecCat+['cluster']].groupby('cluster').agg(lambda x:x.value_counts().index[0])

,t_viajero,STD_cat2_PROMEDIO_SALDO,STD_cat2_PROM_SEGUNDOS,STD_cat2_PROM_DATOS,slope_cat2_PROMEDIO_SALDO,slope_cat2_PROM_SEGUNDOS,slope_cat2_PROM_DATOS,razon_COMERCIAL,razon_COBRO,razon_CANCELACION,razon_INFORMACION,POROUT,res_Region
cluster,,,,,,,,,,,,,
0,sinCambio,nan,moderate,moderate,nan,noTrend,increase,0,0,0,0,0,nan
1,sinInfo,nan,nan,nan,nan,nan,nan,0,0,0,0,0,nan
2,sinCambio,nan,moderate,united,nan,decrease,increase,0,0,0,0,0,SurOccidente
3,sinCambio,nan,united,united,nan,noTrend,decrease,0,0,0,0,0,nan
4,sinCambio,united,united,united,noTrend,noTrend,increase,0,0,0,0,0,NorOccidente


In [76]:
tablaFinalSample[varSelecNum+['cluster']].groupby('cluster').mean()

,part_TOTAL_MB_YOUTUBE,part_TOTAL_MB_WHATSAPP,part_TOTAL_MB_TWITTER_Y_FACEBOOK,part_TOTAL_MB_SNAPCHAT,part_TOTAL_MB_INSTAGRAM,part_TOTAL_MB_GOOGLE_MAPS,CONSUMO_TOTAL_MB,avg_PROMEDIO_SALDO,avg_PROM_SEGUNDOS,avg_PROM_DATOS,paq_youtube_q,paq_ws_q,paq_waze_q,paq_speedy_q,paq_snapchat_q,paq_minutos_q,paq_migracion_q,paq_Instagram_q,paq_facebook_q,paq_datos_q,ANTIGUEDAD_USUARIO,ACTIGUEDAD_ACT_BSCS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES
cluster,,,,,,,,,,,,,,,,,,,,,,,,
0,0.068,0.122,0.313,0.005,0.014,0.010,3553.546,132.468,342.430,5917013.081,0.000,0.417,0.000,0.111,0.000,0.213,0.213,0.000,0.380,0.500,650.546,763.176,-1250.713,0.750
1,0.008,0.017,0.024,0.001,0.001,0.000,93.876,353.821,116.936,190041.010,0.000,0.028,0.000,0.016,0.000,0.075,0.075,0.000,0.023,0.042,482.478,553.781,-211.409,0.132
2,0.042,0.150,0.413,0.007,0.003,0.000,8249.848,624.498,379.263,15332006.853,0.000,0.364,0.000,0.000,0.000,0.242,0.061,0.000,0.364,0.212,654.273,768.697,-1757.576,0.455
3,0.067,0.107,0.354,0.004,0.001,0.000,11699.800,333.120,295.271,28931804.496,0.000,0.100,0.000,0.200,0.000,0.000,0.000,0.000,0.100,0.100,522.250,635.350,-674.500,0.300
4,0.052,0.159,0.510,0.001,0.004,0.000,27894.714,29.311,300.559,56402469.995,0.000,0.143,0.000,0.000,0.000,0.000,0.000,0.000,0.143,0.143,708.857,765.714,-1000.000,0.143


In [78]:
tablaFinalSample['cluster'].value_counts()

1    832
0    108
2     33
3     20
4      7
Name: cluster, dtype: int64

### Ejercicio 3
* Sample
* Seleccion columnas
* Selección Filas solo con info Servicios

In [80]:
varPq=['paq_youtube', 'paq_ws', 'paq_waze', 'paq_twitter','paq_snapchat',
        'paq_sms','paq_Instagram','paq_minutos','paq_datos']
varCon=['CONSUMO_TOTAL_MB','TOTAL_MB_YOUTUBE','TOTAL_MB_WHATSAPP','TOTAL_MB_TWITTER_Y_FACEBOOK','TOTAL_MB_SNAPCHAT',
        'TOTAL_MB_INSTAGRAM','TOTAL_MB_GOOGLE_MAPS']

In [84]:
tablaFinal['selConsumo']=tablaFinal[varCon].max(axis=1)

In [100]:
len(tablaFinal[tablaFinal['selConsumo']>=0])

2303098

In [86]:
tablaFinal['selPaq']=tablaFinal[varPq].max(axis=1)

In [103]:
tablaFinal['selPaq'].value_counts()

0.000    7466615
1.000     795516
Name: selPaq, dtype: int64

In [113]:
#tablaFinal['selCluster']=np.where((tablaFinal['selConsumo']>=0) | (tablaFinal['selPaq']=='1'),1,0)
tablaFinal['selCluster']=np.where(tablaFinal['selConsumo']>=0,1,np.where(tablaFinal['selPaq']==1,1,0))

In [114]:
tablaFinal['selCluster'].value_counts()

0    5819802
1    2442329
Name: selCluster, dtype: int64

In [115]:
#tablaFinal['selClusterV2']=np.where((tablaFinal['selConsumo']>=0)&(tablaFinal['selPaq']=='1'),1,0)
tablaFinal['selClusterV2']=np.where(tablaFinal['selConsumo']>=0,np.where(tablaFinal['selPaq']==1,1,0),0)

In [116]:
tablaFinal['selClusterV2'].value_counts()

0    7605846
1     656285
Name: selClusterV2, dtype: int64

In [117]:
tablaFinalSample=tablaFinal[tablaFinal['selCluster']==1]

In [93]:
kproto = KPrototypes(n_clusters=5, verbose=2, max_iter=20)

In [95]:
cat_idx=np.arange(0,len(varSelecCat),1)
cat_idx=list(cat_idx)

In [96]:
clusters=kproto.fit(tablaFinalSample[varSelecCat+varSelecNum], categorical=cat_idx)

Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 403425, ncost: 4.1964285948297675e+19
Run: 1, iteration: 2/20, moves: 230827, ncost: 3.4528576691029377e+19
Run: 1, iteration: 3/20, moves: 107612, ncost: 3.1225825870940545e+19
Run: 1, iteration: 4/20, moves: 63543, ncost: 2.9423985867676594e+19
Run: 1, iteration: 5/20, moves: 64194, ncost: 2.827323308794728e+19
Run: 1, iteration: 6/20, moves: 65545, ncost: 2.7492076882950554e+19
Run: 1, iteration: 7/20, moves: 61375, ncost: 2.6915787040871113e+19
Run: 1, iteration: 8/20, moves: 5397

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-96-4cb794cdbdb5>", line 1, in <module>
    clusters=kproto.fit(tablaFinalSample[varSelecCat+varSelecNum], categorical=cat_idx)
  File "C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\kmodes\kprototypes.py", line 459, in fit
    self.n_iter_, self.epoch_costs_, self.gamma = k_prototypes(
  File "C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\kmodes\kprototypes.py", line 313, in k_prototypes
    results.append(k_prototypes_single(Xnum, Xcat, nnumattrs, ncatattrs,
  File "C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\kmodes\kprototypes.py", line 248, in k_prototypes_single
    labels, ncost = _labels_cost(Xnum, Xcat, centroids,
  File "C:\ProgramData\Anaconda3\envs\ds_sqdm_claro\lib\site-packages\kmodes\kprot


KeyboardInterrupt



In [ ]:
print(kproto.cluster_centroids_)

In [ ]:
tablaFinalSample['cluster']=clusters.labels_
tablaFinalSample.sample(20)

In [ ]:
tablaFinalSample[varSelecCat+['cluster']].groupby('cluster').agg(lambda x:x.value_counts().index[0])

In [ ]:
tablaFinalSample[varSelecNum+['cluster']].groupby('cluster').mean()

In [ ]:
tablaFinalSample['cluster'].value_counts()

In [ ]:
tablaFinalSample.to_csv('clusterv1.csv')

### Ejercicio 4
* Sample
* Seleccion columnas
* Selección Filas solo con info Servicios
* Sample para reducir tiempo
* Normalización

In [118]:
tablaFinalSample=tablaFinalSample.sample(1000000)

In [122]:
tablaFinalSample.head(10)

,tipo_ACT,t_viajero,STD_cat2_PROMEDIO_SALDO,STD_cat2_PROM_SEGUNDOS,STD_cat2_PROM_DATOS,slope_cat2_PROMEDIO_SALDO,slope_cat2_PROM_SEGUNDOS,slope_cat2_PROM_DATOS,res_Region,res_Mun,res_Depto,razon_VOZ Y DATOS,razon_SERVICIO,razon_PQR,razon_PAGO,razon_INFORMACION,razon_FACTURACION,razon_EQUIPO,razon_COMERCIAL,razon_COBRO,razon_CANCELACION,POROUT,paq_youtube,paq_wtf,paq_ws,paq_waze,paq_twitter,paq_todoIncluido,paq_suscripcion,paq_speedy,paq_snapchat,paq_sms,paq_sinpaquete,paq_pasatiempo,paq_minutos,paq_migracion,paq_LargaDistancia,paq_Instagram,paq_facebook,paq_empresas,paq_datos,ES_SMARTPHONE,TOTAL_VALOR_DE_ADELANTA_SALDO,TOTAL_MB_YOUTUBE,TOTAL_MB_WHATSAPP,TOTAL_MB_TWITTER_Y_FACEBOOK,TOTAL_MB_SNAPCHAT,TOTAL_MB_INSTAGRAM,TOTAL_MB_GOOGLE_MAPS,STD_PROMEDIO_SALDO,STD_PROM_SEGUNDOS,STD_PROM_DATOS,STD_cat_PROMEDIO_SALDO,STD_cat_PROM_SEGUNDOS,STD_cat_PROM_DATOS,slope_PROMEDIO_SALDO,slope_PROM_SEGUNDOS,slope_PROM_DATOS,slope_cat_PROMEDIO_SALDO,slope_cat_PROM_SEGUNDOS,slope_cat_PROM_DATOS,part_TOTAL_MB_YOUTUBE,part_TOTAL_MB_WHATSAPP,part_TOTAL_MB_TWITTER_Y_FACEBOOK,part_TOTAL_MB_SNAPCHAT,part_TOTAL_MB_INSTAGRAM,part_TOTAL_MB_GOOGLE_MAPS,paq_youtube_v,paq_youtube_q,paq_wtf_v,paq_wtf_q,paq_ws_v,paq_ws_q,paq_waze_v,paq_waze_q,paq_twitter_v,paq_twitter_q,paq_todoIncluido_v,paq_todoIncluido_q,paq_suscripcion_v,paq_suscripcion_q,paq_speedy_v,paq_speedy_q,paq_snapchat_v,paq_snapchat_q,paq_sms_v,paq_sms_q,paq_sinpaquete_v,paq_sinpaquete_q,paq_pasatiempo_v,paq_pasatiempo_q,paq_minutos_v,paq_minutos_q,paq_migracion_v,paq_migracion_q,paq_LargaDistancia_v,paq_LargaDistancia_q,paq_Instagram_v,paq_Instagram_q,paq_facebook_v,paq_facebook_q,paq_empresas_v,paq_empresas_q,paq_datos_v,paq_datos_q,CONSUMO_TOTAL_MB,avg_PROMEDIO_SALDO,avg_PROM_SEGUNDOS,avg_PROM_DATOS,ANTIGUEDAD_USUARIO,ACTIGUEDAD_ACT_BSCS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES,selConsumo,selPaq,selCluster,selClusterV2
7648440,seMantiene,sinCambio,united,moderate,united,noTrend,increase,decrease,Costa,Pedraza,Magdalena,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1.0,0.000,118.000,734.000,706.000,103.000,0.000,0.000,0.000,281.309,1411281.739,0.000,0.350,0.385,-0.000,215.521,-998031.427,-0.000,0.268,-0.272,0.040,0.250,0.240,0.035,0.000,0.000,-0.000,0.000,-0.000,0.000,-2000.000,1.000,-0.000,0.000,-2000.000,1.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-2000.000,1.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-2000.000,1.000,-0.000,0.000,-2000.000,1.000,2941.000,1.300,803.904,3670186.851,860.000,893,-2000.000,1.000,2941.000,1.000,1,1
2012266,seMantiene,sinCambio,moderate,united,united,decrease,increase,increase,Costa,Barranquilla,Atlantico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.000,477.000,3206.000,4367.000,5.000,0.000,0.000,6.442,32.917,1416260.416,0.979,0.255,0.117,-5.935,16.069,1415737.182,-0.902,0.124,0.117,0.049,0.331,0.451,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9691.000,6.580,129.319,12111052.061,609.000,673,0.000,0.000,9691.000,0.000,1,0
8124152,seMantiene,sinCambio,dispersed,dispersed,moderate,increase,increase,increase,Costa,Villanueva,Bolivar,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1.0,0.000,0.000,0.000,83.000,0.000,0.000,0.000,159.524,326.076,1101021.582,1.673,0.549,1.106,138.097,87.752,672556.185,1.448,0.148,0.675,0.000,0.000,0.686,0.000,0.000,0.000,-0.000,0.000,-0.000,0.000,-4000.000,1.000,-0.000,0.000,-4000.000,1.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-4000.000,1.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-4000.000,1.000,-0.000,0.000,-4000.000,1.000,121.000,95.346,594.147,995675.956,711.000,715,-4000.000,1.000,121.000,1.000,1,1
7726561,seMantiene,sinCambio,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,0,0,

In [123]:
data_to_standardize = tablaFinalSample[varSelecNum]
scaler = StandardScaler().fit(data_to_standardize)

In [125]:
# Standardize the columns.
standardized_data = tablaFinalSample.copy()
standardized_columns = scaler.transform(data_to_standardize)
standardized_data[varSelecNum] = standardized_columns

In [126]:
standardized_data.head(10)

,tipo_ACT,t_viajero,STD_cat2_PROMEDIO_SALDO,STD_cat2_PROM_SEGUNDOS,STD_cat2_PROM_DATOS,slope_cat2_PROMEDIO_SALDO,slope_cat2_PROM_SEGUNDOS,slope_cat2_PROM_DATOS,res_Region,res_Mun,res_Depto,razon_VOZ Y DATOS,razon_SERVICIO,razon_PQR,razon_PAGO,razon_INFORMACION,razon_FACTURACION,razon_EQUIPO,razon_COMERCIAL,razon_COBRO,razon_CANCELACION,POROUT,paq_youtube,paq_wtf,paq_ws,paq_waze,paq_twitter,paq_todoIncluido,paq_suscripcion,paq_speedy,paq_snapchat,paq_sms,paq_sinpaquete,paq_pasatiempo,paq_minutos,paq_migracion,paq_LargaDistancia,paq_Instagram,paq_facebook,paq_empresas,paq_datos,ES_SMARTPHONE,TOTAL_VALOR_DE_ADELANTA_SALDO,TOTAL_MB_YOUTUBE,TOTAL_MB_WHATSAPP,TOTAL_MB_TWITTER_Y_FACEBOOK,TOTAL_MB_SNAPCHAT,TOTAL_MB_INSTAGRAM,TOTAL_MB_GOOGLE_MAPS,STD_PROMEDIO_SALDO,STD_PROM_SEGUNDOS,STD_PROM_DATOS,STD_cat_PROMEDIO_SALDO,STD_cat_PROM_SEGUNDOS,STD_cat_PROM_DATOS,slope_PROMEDIO_SALDO,slope_PROM_SEGUNDOS,slope_PROM_DATOS,slope_cat_PROMEDIO_SALDO,slope_cat_PROM_SEGUNDOS,slope_cat_PROM_DATOS,part_TOTAL_MB_YOUTUBE,part_TOTAL_MB_WHATSAPP,part_TOTAL_MB_TWITTER_Y_FACEBOOK,part_TOTAL_MB_SNAPCHAT,part_TOTAL_MB_INSTAGRAM,part_TOTAL_MB_GOOGLE_MAPS,paq_youtube_v,paq_youtube_q,paq_wtf_v,paq_wtf_q,paq_ws_v,paq_ws_q,paq_waze_v,paq_waze_q,paq_twitter_v,paq_twitter_q,paq_todoIncluido_v,paq_todoIncluido_q,paq_suscripcion_v,paq_suscripcion_q,paq_speedy_v,paq_speedy_q,paq_snapchat_v,paq_snapchat_q,paq_sms_v,paq_sms_q,paq_sinpaquete_v,paq_sinpaquete_q,paq_pasatiempo_v,paq_pasatiempo_q,paq_minutos_v,paq_minutos_q,paq_migracion_v,paq_migracion_q,paq_LargaDistancia_v,paq_LargaDistancia_q,paq_Instagram_v,paq_Instagram_q,paq_facebook_v,paq_facebook_q,paq_empresas_v,paq_empresas_q,paq_datos_v,paq_datos_q,CONSUMO_TOTAL_MB,avg_PROMEDIO_SALDO,avg_PROM_SEGUNDOS,avg_PROM_DATOS,ANTIGUEDAD_USUARIO,ACTIGUEDAD_ACT_BSCS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES,selConsumo,selPaq,selCluster,selClusterV2
7648440,seMantiene,sinCambio,united,moderate,united,noTrend,increase,decrease,Costa,Pedraza,Magdalena,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1.0,0.000,118.000,734.000,706.000,103.000,0.000,0.000,0.000,281.309,1411281.739,0.000,0.350,0.385,-0.000,215.521,-998031.427,-0.000,0.268,-0.272,-0.191,0.751,-0.079,1.932,-0.200,-0.085,-0.000,-0.011,-0.000,0.000,-2000.000,0.360,-0.000,-0.012,-2000.000,1.000,-0.000,0.000,-0.000,0.000,-0.000,-0.135,-0.000,-0.005,-0.000,0.000,-2000.000,1.000,-0.000,0.000,-0.000,-0.202,-0.000,-0.200,-0.000,0.000,-0.000,-0.011,-2000.000,0.546,-0.000,0.000,-2000.000,0.257,-0.088,-0.122,1.673,-0.229,0.243,0.130,-0.092,-0.009,2941.000,1.000,1,1
2012266,seMantiene,sinCambio,moderate,united,united,decrease,increase,increase,Costa,Barranquilla,Atlantico,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.000,477.000,3206.000,4367.000,5.000,0.000,0.000,6.442,32.917,1416260.416,0.979,0.255,0.117,-5.935,16.069,1415737.182,-0.902,0.124,0.117,-0.116,1.246,0.691,-0.210,-0.200,-0.085,0.000,-0.011,0.000,0.000,0.000,-0.275,0.000,-0.012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.135,0.000,-0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.202,0.000,-0.200,0.000,0.000,0.000,-0.011,0.000,-0.254,0.000,0.000,0.000,-0.301,0.836,-0.121,-0.684,0.614,-0.116,-0.177,0.360,-0.362,9691.000,0.000,1,0
8124152,seMantiene,sinCambio,dispersed,dispersed,moderate,increase,increase,increase,Costa,Villanueva,Bolivar,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1.0,0.000,0.000,0.000,83.000,0.000,0.000,0.000,159.524,326.076,1101021.582,1.673,0.549,1.106,138.097,87.752,672556.185,1.448,0.148,0.675,-0.521,-0.768,1.552,-0.242,-0.200,-0.085,-0.000,-0.011,-0.000,0.000,-4000.000,0.360,-0.000,-0.012,-4000.000,1.000,-0.000,0.000,-0.000,0.000,-0.000,-0.135,-0.000,-0.005,-0.000,0.000,-4000.000,1.000,-0.000,0.000,-0.000,-0.202,-0.000,-0.200,-0.000,0.000,-0.000,-0.011,-4000.000,0.546,-0.000,0.000,-4000.000,0.257,-0.475,-0.104,0.940,-0.496,0.030,-0.119,-0.544,-0.009,121.000,1.000,1,1
7726561,seMantiene,sinCambio,nan,nan,nan,nan,nan,nan,nan,nan,

In [127]:
kproto = KPrototypes(n_clusters=5, verbose=2, max_iter=20)

In [128]:
cat_idx=np.arange(0,len(varSelecCat),1)
cat_idx=list(cat_idx)

In [130]:
clusters=kproto.fit(standardized_data[varSelecCat+varSelecNum], categorical=cat_idx)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 234489, ncost: 21548216.14207047
Run: 1, iteration: 2/20, moves: 189290, ncost: 21117243.350211363
Run: 1, iteration: 3/20, moves: 124452, ncost: 20969786.512962256
Run: 1, iteration: 4/20, moves: 57128, ncost: 20931836.59227825
Run: 1, iteration: 5/20, moves: 27830, ncost: 20919415.40780525
Run: 1, iteration: 6/20, moves: 14717, ncost: 20915487.016590532
Run: 1, iteration: 7/20, moves: 8016, ncost: 20914252.461423684
Run: 1, iteration: 8/20, moves: 4494, ncost: 20913884.193605807
Run: 1, iteration: 9/20, moves: 2541, ncost: 20913773.208373923
Run: 1, iteration: 10/20, moves: 1369, ncost: 20913742.444004897
Run: 1, iteration: 11/20, moves: 758, ncost: 20913731.7585925
Run: 1, iteration: 12/20, moves: 446, ncost: 20913728.433801588
Run: 1, iteration: 13/20, moves: 225, ncost: 20913727.62970411
Run: 1, iteration: 14/20, moves: 88, ncost: 20913727.486199692
Run: 1, iteration: 15

Run: 6, iteration: 20/20, moves: 334, ncost: 21546213.247673064
Run: 6, iteration: 21/20, moves: 226, ncost: 21546212.148045097
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 7, iteration: 1/20, moves: 297332, ncost: 20770439.74113675
Run: 7, iteration: 2/20, moves: 111791, ncost: 20300528.566153552
Run: 7, iteration: 3/20, moves: 45049, ncost: 20275287.848892804
Run: 7, iteration: 4/20, moves: 17820, ncost: 20271098.340817895
Run: 7, iteration: 5/20, moves: 10970, ncost: 20268777.841447663
Run: 7, iteration: 6/20, moves: 11162, ncost: 20264738.825100165
Run: 7, iteration: 7/20, moves: 12467, ncost: 20263831.927703694
Run: 7, iteration: 8/20, moves: 5187, ncost: 20263396.729537513
Run: 7, iteration: 9/20, moves: 3653, ncost: 20263195.99767183
Run: 7, iteration: 10/20, moves: 2652, ncost: 20263098.929414432
Run: 7, iteration: 11/20, moves: 1889, ncost: 20263049.225168347
Run: 7, iteration: 12/20, moves: 1371, ncost: 20263025.418143887
Run: 7, iterat

In [138]:
print(kproto.cluster_centroids_)

[['-0.2400967225445031' '-0.4792174390571895' '-0.6001356042480194'
  '-0.14160401606247236' '-0.14524609797916796' '-0.05549496301758715'
  '-0.4303079042148574' '-0.038856874853992224' '0.22534663928058205'
  '-0.4326419204284412' '-0.011175960140499484' '-0.2647364927514429'
  '-0.012375718695924237' '-0.05547914625166322' '-0.004880993442504471'
  '6.987345169288751' '6.98420589055018' '-0.010837767453241226'
  '-0.2537693247764132' '-0.18053480566930835' '0.22176022333005246'
  '0.19080690914402584' '-3.4024540599932585' '5.492044954420831'
  'sinCambio' 'dispersed' 'united' 'nan' 'increase' 'noTrend' 'nan' '0'
  '0' '0' '0' '0' 'Costa']
 ['-0.06803272415191107' '0.5815464758685884' '0.3588284043883614'
  '0.010836192013669844' '0.2776208924947874' '0.057634692580819775'
  '0.08251904117343851' '0.14982341288037807' '-0.20820817994589585'
  '0.11376718376657402' '0.015169512223723067' '4.298595258561683'
  '0.06936442937966773' '0.32142032205968396' '0.0114832838470741'
  '0.09091

In [139]:
tablaFinalSample['cluster']=clusters.labels_
tablaFinalSample.sample(20)

,tipo_ACT,t_viajero,STD_cat2_PROMEDIO_SALDO,STD_cat2_PROM_SEGUNDOS,STD_cat2_PROM_DATOS,slope_cat2_PROMEDIO_SALDO,slope_cat2_PROM_SEGUNDOS,slope_cat2_PROM_DATOS,res_Region,res_Mun,res_Depto,razon_VOZ Y DATOS,razon_SERVICIO,razon_PQR,razon_PAGO,razon_INFORMACION,razon_FACTURACION,razon_EQUIPO,razon_COMERCIAL,razon_COBRO,razon_CANCELACION,POROUT,paq_youtube,paq_wtf,paq_ws,paq_waze,paq_twitter,paq_todoIncluido,paq_suscripcion,paq_speedy,paq_snapchat,paq_sms,paq_sinpaquete,paq_pasatiempo,paq_minutos,paq_migracion,paq_LargaDistancia,paq_Instagram,paq_facebook,paq_empresas,paq_datos,ES_SMARTPHONE,TOTAL_VALOR_DE_ADELANTA_SALDO,TOTAL_MB_YOUTUBE,TOTAL_MB_WHATSAPP,TOTAL_MB_TWITTER_Y_FACEBOOK,TOTAL_MB_SNAPCHAT,TOTAL_MB_INSTAGRAM,TOTAL_MB_GOOGLE_MAPS,STD_PROMEDIO_SALDO,STD_PROM_SEGUNDOS,STD_PROM_DATOS,STD_cat_PROMEDIO_SALDO,STD_cat_PROM_SEGUNDOS,STD_cat_PROM_DATOS,slope_PROMEDIO_SALDO,slope_PROM_SEGUNDOS,slope_PROM_DATOS,slope_cat_PROMEDIO_SALDO,slope_cat_PROM_SEGUNDOS,slope_cat_PROM_DATOS,part_TOTAL_MB_YOUTUBE,part_TOTAL_MB_WHATSAPP,part_TOTAL_MB_TWITTER_Y_FACEBOOK,part_TOTAL_MB_SNAPCHAT,part_TOTAL_MB_INSTAGRAM,part_TOTAL_MB_GOOGLE_MAPS,paq_youtube_v,paq_youtube_q,paq_wtf_v,paq_wtf_q,paq_ws_v,paq_ws_q,paq_waze_v,paq_waze_q,paq_twitter_v,paq_twitter_q,paq_todoIncluido_v,paq_todoIncluido_q,paq_suscripcion_v,paq_suscripcion_q,paq_speedy_v,paq_speedy_q,paq_snapchat_v,paq_snapchat_q,paq_sms_v,paq_sms_q,paq_sinpaquete_v,paq_sinpaquete_q,paq_pasatiempo_v,paq_pasatiempo_q,paq_minutos_v,paq_minutos_q,paq_migracion_v,paq_migracion_q,paq_LargaDistancia_v,paq_LargaDistancia_q,paq_Instagram_v,paq_Instagram_q,paq_facebook_v,paq_facebook_q,paq_empresas_v,paq_empresas_q,paq_datos_v,paq_datos_q,CONSUMO_TOTAL_MB,avg_PROMEDIO_SALDO,avg_PROM_SEGUNDOS,avg_PROM_DATOS,ANTIGUEDAD_USUARIO,ACTIGUEDAD_ACT_BSCS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES,selConsumo,selPaq,selCluster,selClusterV2,cluster
1771502,seMantiene,sinCambio,dispersed,moderate,united,increase,increase,decrease,Centro,Guasca,Cundinamarca,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,676.824,74.100,176857.669,1.727,0.392,0.461,586.150,40.068,-174377.184,1.496,0.212,-0.454,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37.000,391.825,189.248,383784.268,701.000,773,0.000,0.000,37.000,0.000,1,0,4
1252611,seMantiene,sinCambio,moderate,united,united,noTrend,decrease,decrease,SurOccidente,Colon,Putumayo,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.000,1.000,1.000,52.000,0.000,0.000,0.000,525.819,27.670,648501.443,0.866,0.165,0.521,54.167,-17.998,-242948.834,0.089,-0.107,-0.195,0.016,0.016,0.839,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,62.000,607.383,168.119,1244326.327,992.000,1023,0.000,0.000,62.000,0.000,1,0,2
2725018,seMantiene,sinCambio,united,moderate,moderate,noTrend,decrease,increase,Oriente,Cerrito,Santander,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.000,537.000,155.000,25.000,0.000,0.000,0.000,0.000,99.742,7330261.991,0.000,0.382,0.714,-0.000,-48.479,5513393.831,-0.000,-0.186,0.537,0.446,0.129,0.021,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1204.000,1.000,260.835,10266750.065,361.000,431,0.000,0.000,1204.000,0.000,1,0,4
1195454,seMantiene,sinCambio,united,moderate,united,noTrend,decrease,increase,SurOccidente,San Andres De Tumaco,Nari�o,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [148]:
clusTotalCat=tablaFinalSample[varSelecCat+['cluster']].groupby('cluster').agg(lambda x:x.value_counts().index[0])
clusTotalCat

,t_viajero,STD_cat2_PROMEDIO_SALDO,STD_cat2_PROM_SEGUNDOS,STD_cat2_PROM_DATOS,slope_cat2_PROMEDIO_SALDO,slope_cat2_PROM_SEGUNDOS,slope_cat2_PROM_DATOS,razon_COMERCIAL,razon_COBRO,razon_CANCELACION,razon_INFORMACION,POROUT,res_Region
cluster,,,,,,,,,,,,,
0,sinCambio,dispersed,united,nan,increase,noTrend,nan,0,0,0,0,0,Costa
1,sinCambio,moderate,moderate,united,increase,noTrend,increase,0,0,0,0,0,Oriente
2,sinCambio,united,united,united,noTrend,noTrend,increase,0,0,0,0,0,NorOccidente
3,sinCambio,nan,united,united,nan,noTrend,increase,0,0,0,0,0,Oriente
4,sinCambio,nan,moderate,moderate,nan,noTrend,increase,0,0,0,0,0,nan


In [153]:
clusTotalCat.to_csv('clusterCat.csv',sep=';')

In [154]:
varSelecNum2=[
 'paq_youtube_v', 'paq_ws_v', 'paq_waze_v',  'paq_speedy_v', 'paq_snapchat_v', 'paq_minutos_v',
 'paq_migracion_v', 'paq_Instagram_v', 'paq_facebook_v', 'paq_datos_v']

In [160]:
clusTotalNum=tablaFinalSample[varSelecNum+['cluster']+varSelecNum2].groupby('cluster').mean()
clusTotalNum

,part_TOTAL_MB_YOUTUBE,part_TOTAL_MB_WHATSAPP,part_TOTAL_MB_TWITTER_Y_FACEBOOK,part_TOTAL_MB_SNAPCHAT,part_TOTAL_MB_INSTAGRAM,part_TOTAL_MB_GOOGLE_MAPS,CONSUMO_TOTAL_MB,avg_PROMEDIO_SALDO,avg_PROM_SEGUNDOS,avg_PROM_DATOS,paq_youtube_q,paq_ws_q,paq_waze_q,paq_speedy_q,paq_snapchat_q,paq_minutos_q,paq_migracion_q,paq_Instagram_q,paq_facebook_q,paq_datos_q,ANTIGUEDAD_USUARIO,ACTIGUEDAD_ACT_BSCS,VLR_PAQ_MAS_COMPRADO_2MESES,CANT_PAQ_MAS_COMPRADO_2MESES,paq_youtube_v,paq_ws_v,paq_waze_v,paq_speedy_v,paq_snapchat_v,paq_minutos_v,paq_migracion_v,paq_Instagram_v,paq_facebook_v,paq_datos_v
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.034,0.047,0.098,0.002,0.002,0.000,445.741,445.212,389.514,1634920.972,0.000,0.016,0.000,0.049,0.000,16.591,16.508,0.000,0.000,0.216,845.319,936.319,-16653.089,16.598,0.000,-15.477,0.000,-61.757,0.000,-16647.692,-16503.744,0.000,0.000,-218.093
1,0.055,0.222,0.360,0.004,0.021,0.002,4189.114,1456.820,265.425,7105074.165,0.001,7.206,0.003,0.283,0.000,0.675,0.620,0.003,5.017,8.208,750.558,858.041,-14806.701,8.382,-2.112,-13033.247,-6.299,-579.903,-0.321,-1306.931,-903.085,-7.355,-10641.073,-14130.940
2,0.062,0.168,0.257,0.003,0.005,0.001,1818.498,985.484,396.335,3466280.625,0.000,0.255,0.001,0.143,0.000,0.271,0.265,0.000,0.199,0.407,1587.105,1722.169,-1188.199,0.757,-0.674,-536.435,-0.696,-228.228,-0.085,-347.252,-311.562,-0.833,-465.786,-696.801
3,0.054,0.127,0.592,0.005,0.008,0.001,16632.815,279.847,429.357,23184184.834,0.001,0.284,0.000,0.140,0.000,0.085,0.077,0.001,0.236,0.336,527.902,624.923,-1036.226,0.512,-2.229,-689.852,-0.677,-211.216,-0.205,-150.838,-93.294,-2.475,-628.067,-760.390
4,0.067,0.108,0.189,0.004,0.010,0.001,1381.013,583.059,276.574,3105510.065,0.000,0.244,0.000,0.041,0.000,0.256,0.253,0.000,0.180,0.303,388.205,491.672,-954.156,0.571,-0.582,-519.580,-0.419,-62.972,-0.077,-343.458,-318.180,-0.514,-440.016,-588.383


In [158]:
clusTotalNum.to_csv('clusterNum.csv',sep=';')

In [142]:
tablaFinalSample['cluster'].value_counts()

4    602757
2    223232
3    134156
1     26513
0     13342
Name: cluster, dtype: int64

In [143]:
tablaFinalSample.to_csv('clusterv2.csv')

In [137]:
###  guardar dump sobre el cluster para futuras predicciones

In [144]:
dump(kproto,'modelosProto.kpl')

['modelosProto.kpl']

In [145]:
#futuras preducciones
clusters2 = kproto.predict(standardized_data.sample(5)[varSelecCat+varSelecNum], categorical=cat_idx)

In [146]:
clusters2

array([4, 4, 4, 4, 1], dtype=uint16)

###  Indice Porout por Cluster

In [162]:
tablaFinalSample[['cluster','POROUT']].value_counts()

cluster  POROUT
4        0         545845
2        0         198163
3        0         118994
4        1          56912
2        1          25069
1        0          22591
3        1          15162
0        0          12346
1        1           3922
0        1            996
dtype: int64

### proximos pasos
* Eliminar valores anómalos de variables numéricas

#### Pruebas
* Datos sin NA
* Seleccionar menos variables
* Medias de los datos numericos

In [ ]:
tablaFinalSample.groupby('cluster').agg(
    {
        'Total':'count',
        'Region': lambda x: x.value_counts().index[0],
        'Item Type': lambda x: x.value_counts().index[0],
        'Sales Channel': lambda x: x.value_counts().index[0],
        'Order Priority': lambda x: x.value_counts().index[0],
        'Units Sold': 'mean',
        'Unit Price': 'mean',
        'Total Revenue': 'mean',
        'Total Cost': 'mean',
        'Total Profit': 'mean'
    }
).reset_index()

In [ ]:
# Cluster interpretation
df.rename(columns = {'Cluster Labels':'Total'}, inplace = True)
df.groupby('Segment').agg(
    {
        'Total':'count',
        'Region': lambda x: x.value_counts().index[0],
        'Item Type': lambda x: x.value_counts().index[0],
        'Sales Channel': lambda x: x.value_counts().index[0],
        'Order Priority': lambda x: x.value_counts().index[0],
        'Units Sold': 'mean',
        'Unit Price': 'mean',
        'Total Revenue': 'mean',
        'Total Cost': 'mean',
        'Total Profit': 'mean'
    }
).reset_index()